<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/On_the_Use_of_Cross_Domain_User_Preferences_and_Personality_Traits_in_Collaborative_Filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# On the Use of Cross-Domain User Preferences and Personality Traits in Collaborative Filtering
# Ignacio Fernández-Tobías and Iván Cantador

In [ ]:
import pandas as pd
import scipy, cmake
import numpy as np
import warnings
from sklearn import svm
warnings.filterwarnings('ignore')

In [ ]:
# Tripadvisor review Dataset
path = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/movies_pers_num.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  df = pd.read_csv(infile)
arr = df[['0', '1', '2', '3', '4']]
arr.columns = ['open', 'cons', 'extra', 'agree', 'neuro']

In [ ]:
# amazon review dataset musics json
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Movies_and_TV.json'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_json(infile, lines=True, nrows=2400)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Magazine_Subscriptions.csv'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_csv(infile, nrows=1000)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
arr.shape
arr = arr[0:2400]

In [ ]:
src = az
src_pers = arr2
tgt = am
tgt_pers = arr1

In [ ]:
src = az[['userId', 'itemId', 'rating']]

In [ ]:
arr['userId'] = az['userId']
arr['itemId'] = np.array(az['itemId'])
arr['rating'] = az['rating']

# Hu&Pu

In [ ]:
# params = alpha, neighbors

In [ ]:
# rating matrix wrt user u
ru = arr.pivot_table(index='userId',columns='itemId',values='rating')
ru = ru.fillna(0)
ru_m = ru > 0
ru_m = ru_m.replace(True, 1)
ru_m = ru_m.replace(False, 0)
ru = np.array(ru)
ru_m = np.array(ru_m)

In [ ]:
ru.shape

(2386, 22)

In [ ]:
items = []    # U
for i in range(len(ru_m)):
  row = []
  for j in range(len(ru_m[i])):
    if ru_m[i][j] == 1:
      row.append(j)
  items.append(row)
ru_b = np.sum(ru, axis=1) / len(items)

In [ ]:
src = arr

In [ ]:
arr = arr.groupby(by='userId').mean().reset_index()
arr = arr[["open", 'cons', 'extra', 'agree', 'neuro']]

In [ ]:
arr.shape

(2386, 5)

In [ ]:
p_b = np.sum(arr, axis=1) / len(arr)

In [ ]:
ru_b = ru_b.reshape([len(ru_b), 1])
ru_s = ru-ru_b
ru_ss = np.sum(ru-ru_b, axis=1)
ru_sy = np.square(ru_s)
ru_sy = np.sum(ru_sy, axis=1)

In [ ]:
ru_ss.shape

(2386,)

In [ ]:
simr = []
simp = []
for u in range(len(ru)):
  row1=[]
  for v in range(len(ru)):
    x = np.multiply(ru_ss[u], ru_ss[v])
    y = np.multiply(ru_sy[u], ru_sy[v])
    row1.append(x/y)
  simr.append(row1)

In [ ]:
p_b = np.array(p_b).reshape([len(p_b), 1])

In [ ]:
arr = np.array(arr)

In [ ]:
simp = []
for u in range(len(arr)):
  row = []
  for v in range(len(arr)):
      x = np.multiply((arr[u] - p_b[u]), (arr[v] - p_b[v]))
      y = np.multiply(np.sum(np.square(arr[u] - p_b[u])), (np.square(arr[v] - p_b[v])))
      row.append(np.sum(x/y))
  simp.append(row)

In [ ]:
simp = np.array(simp)
simr = np.array(simr)

In [ ]:
arr.shape

(2386, 5)

In [ ]:
alpha = 0.6

sim_uv = alpha * simp + (1-alpha) * simr

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=30, algorithm='ball_tree').fit(arr)

In [ ]:
distances, indices = knn.kneighbors(arr)

In [ ]:
rating_pred = []
for u in range(len(ru_m)):
  row = []
  for i in range(len(ru_m[u])):
    sum = 0
    all_n = indices[u]
    users_taking_i = ru_m[:, i]   # Nu
    for v in range(len(users_taking_i)):
        sum = sum + np.sum(sim_uv[u][v] * users_taking_i)
        if sum > 5:
          sum = 5 
    row.append(sum)
  rating_pred.append(row)

In [ ]:
rating_pred = np.array(rating_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(ru, rating_pred)
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(ru, rating_pred)
print(mae, rmse)

4.79055856130458 23.90524270365008


In [ ]:
#tuning

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
alpha_arr = [0.3, 0.4, 0.5, 0.7, 0.8, 0.9]
nn = [5,10,15,20,50]

In [ ]:
ru.shape

(284, 8933)

In [ ]:
for a in alpha_arr:
  for k in nn:
    alpha = a
    sim_uv = alpha * simp + (1-alpha) * simr
    knn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(arr)
    distances, indices = knn.kneighbors(arr)
    rating_pred = []
    for u in range(len(ru_m)):
      row = []
      for i in range(len(ru_m[u])):
        sum = 0
        all_n = indices[u]
        users_taking_i = ru_m[:, i]   # Nu
        for v in range(len(users_taking_i)):
            sum = sum + np.sum(sim_uv[u][v] * users_taking_i)
            if sum > 5:
              sum = 5 
        row.append(sum)
      rating_pred.append(row)
    rating_pred = np.array(rating_pred)
    mae = mean_absolute_error(ru, rating_pred)
    rmse = mean_squared_error(ru, rating_pred)
    print("alpha", a, "neighbours", k)
    print("mae", mae)
    print("rmse", rmse)
    print('----------------------------------------')

# tobias

In [ ]:
src = src.reset_index()
tgt = tgt.reset_index()

In [ ]:
src2 = src.groupby(by='userId').mean().reset_index()
tgt2 = tgt.groupby(by='userId').mean().reset_index()

In [ ]:
common_src = []       # NU
common_tgt = []
common_src_i = []     # NU
common_tgt_i = []
common_users = set([value for value in np.array(src['userId']) if value in np.array(tgt['userId'])])

for i in common_users:
    for j in range(len(src)):
      row = []
      if i == src['userId'][j]:
        common_src_i.append(src['userId'][j])
        row.append(src['index'][j])
        row.append(src['userId'][j])
        row.append(src['itemId'][j])
        row.append(src['rating'][j])
        common_src.append(row)
    for j in range(len(tgt)):
      row = []
      if i == tgt['userId'][j]:
        common_tgt_i.append(tgt['userId'][j])
        row.append(tgt['index'][j])
        row.append(tgt['userId'][j])
        row.append(tgt['itemId'][j])
        row.append(tgt['rating'][j])
        common_tgt.append(row)

In [ ]:
common_tgt

In [ ]:
common_src_i = set(common_src_i)   # U
common_tgt_i = set(common_tgt_i)   # U

In [ ]:
src_df_r = pd.DataFrame(common_src, columns = ['index', 'userId','itemId','rating'])   #NU
src_df_r
tgt_df_r = pd.DataFrame(common_tgt, columns = ['index', 'userId','itemId','rating'])   #NU
tgt_df_r

In [ ]:
# rating matrix wrt user u
ru_src = src_df_r.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_src = ru_src.fillna(0)
ru_m_src = ru_src > 0
ru_m_src = ru_m_src.replace(True, 1)
ru_m_src = ru_m_src.replace(False, 0)
ru_src = np.array(ru_src)
ru_m_src = np.array(ru_m_src)   # U


# rating matrix wrt user u
ru_tgt = tgt_df_r.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_tgt = ru_tgt.fillna(0)
ru_m_tgt = ru_tgt > 0
ru_m_tgt = ru_m_tgt.replace(True, 1)
ru_m_tgt = ru_m_tgt.replace(False, 0)
ru_tgt = np.array(ru_tgt)
ru_m_tgt = np.array(ru_m_tgt)   # U

In [ ]:
len(ru_tgt)

49

In [ ]:
items_src = []    # U
for i in range(len(ru_m_src)):
  row = []
  for j in range(len(ru_m_src[i])):
    if ru_m_src[i][j] == 1:
      row.append(j)
  items_src.append(row)

items_tgt = []   # U
for i in range(len(ru_m_tgt)):
  row = []
  for j in range(len(ru_m_tgt[i])):
    if ru_m_tgt[i][j] == 1:
      row.append(j)
  items_tgt.append(row)

In [ ]:
len(src_df_r)

63

In [ ]:
def intersect(X, Y):
  Z = [value for value in X if value in Y]
  return len(Z)

def union(X, Y):
    Z = X + Y
    return len(set(Z))

In [ ]:
  df = pd.DataFrame(src_pers[0])  # NU
  df.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
  df['userId'] = src.userId
  df2 = df.groupby(by='userId').mean().reset_index()
  df2 = df2[['userId', "open", 'cons', 'extra', 'agree', 'neuro']] 

In [ ]:
df

In [ ]:
  from sklearn.metrics import pairwise_distances 
  lambda_s = 0.67 

  arr = []
  src_arr = []
  for j in range(len(src_df_r)):   ##  NU
    x = src_df_r['userId'][j]  
    y = df[df['userId']==x]
    arr.append(np.array(y)[0][0])
    arr.append(np.array(y)[0][1])
    arr.append(np.array(y)[0][2])
    arr.append(np.array(y)[0][3])
    arr.append(np.array(y)[0][4])
    arr.append(np.array(y)[0][5])
    src_arr.append(arr)    ### NU
    arr = []
  df_t = pd.DataFrame(src_arr)
  src_arr = np.array(df_t.drop_duplicates())    ### U

  df_t = tgt_df_r.pivot_table(index='userId',columns='itemId',values='rating').reset_index()
  df_t = df_t.fillna(0)
  simr_s = []
  for u in range(len(ru_src)):   # U
    row = []
    for v in range(len(ru_src)):    #  U
      isu = items_src[u]
      isv = items_src[v]
      su_n = src_arr[u][5]
      sv_n = src_arr[v][5]
      if su_n in common_users and sv_n in common_users:  # U
        tu_n = np.array(df_t[df_t['userId']==su_n].reset_index())[0][0]   ### U
        tv_n = np.array(df_t[df_t['userId']==sv_n].reset_index())[0][0]   ### U
        itu = items_tgt[tu_n]   # U
        itv = items_tgt[tv_n]
        sp = (intersect(isu, isv) + intersect(itu, itv)) / (union(isu, isv) + union(itu, itv))
        row.append(sp)
    simr_s.append(row)
  simr_s = np.array(simr_s) 

  src_arr = np.delete(src_arr, 5, 1)
  simp_s = 1 - pairwise_distances(src_arr, metric = "hamming") 

  suv_s = lambda_s * simr_s + (1-lambda_s) * simp_s

In [ ]:
for u in range(len(ru_tgt)):
  for i in range(len(ru_tgt[u])):
    

(49, 49)

'A11XKY4EIU2KNR'

In [ ]:
len(items_tgt)

49

# tobias 2

In [ ]:
df = pd.DataFrame(tgt_pers[0])  # NU
df.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
df['userId'] = tgt.userId
df2 = df.groupby(by='userId').mean().reset_index()
df2 = df2[["open", 'cons', 'extra', 'agree', 'neuro']] 
tgt_p_arr = np.array(df2)

In [ ]:
common_src = []       # NU
common_tgt = []
common_src_i = []     # NU
common_tgt_i = []
common_users = set([value for value in np.array(src['userId']) if value in np.array(src['userId'])])

In [ ]:
# rating matrix wrt user u
ru_src = src.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_src = ru_src.fillna(0)
ru_m_src = ru_src > 0
ru_m_src = ru_m_src.replace(True, 1)
ru_m_src = ru_m_src.replace(False, 0)
ru_src = np.array(ru_src)
ru_m_src = np.array(ru_m_src)   # U


# rating matrix wrt user u
ru_tgt = tgt.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_tgt = ru_tgt.fillna(0)
ru_m_tgt = ru_tgt > 0
ru_m_tgt = ru_m_tgt.replace(True, 1)
ru_m_tgt = ru_m_tgt.replace(False, 0)
ru_tgt = np.array(ru_tgt)
ru_m_tgt = np.array(ru_m_tgt)   # U

In [ ]:
items_src = []    # U
for i in range(len(ru_m_src)):
  row = []
  for j in range(len(ru_m_src[i])):
    if ru_m_src[i][j] == 1:
      row.append(j)
  items_src.append(row)

items_tgt = []   # U
for i in range(len(ru_m_tgt)):
  row = []
  for j in range(len(ru_m_tgt[i])):
    if ru_m_tgt[i][j] == 1:
      row.append(j)
  items_tgt.append(row)

In [ ]:
df_s = src.pivot_table(index='userId',columns='itemId',values='rating').reset_index()
df_t = tgt.pivot_table(index='userId',columns='itemId',values='rating').reset_index()

In [ ]:
def intersect(X, Y):
  Z = [value for value in X if value in Y]
  return len(Z)

def union(X, Y):
    Z = X + Y
    return len(set(Z))

In [ ]:
from sklearn.metrics import pairwise_distances 
lambda_s = 0.5

simr = []
for u in range(len(ru_m_tgt)):
  row = []
  for v in range(len(ru_m_tgt)):
    itu = items_tgt[u]
    itv = items_tgt[v]
    tun = df_t['userId'][u]
    tvn = df_t['userId'][v]
    if tun in common_users and tvn in common_users:
      sun = np.array(df_s[df_s['userId']==tun].reset_index())[0][0]
      svn = np.array(df_s[df_s['userId']==tvn].reset_index())[0][0]
      isu = items_src[sun]
      isv = items_src[svn]
      sp = (intersect(itu, itv) + intersect(isu, isv)) / (union(itu, itv) + union(itu, itv))
      row.append(sp)
    else:
      sp = intersect(itu, itv) / union(itu, itv)
      row.append(sp)
    simr.append(row)
    
simp = 1 - pairwise_distances(tgt_p_arr, metric = "hamming")

simr = np.array(simr)
simp = np.array(simp)


suv_t = lambda_s * simr + (1-lambda_s) * simp


In [ ]:
rating_pred = []
for u in range(len(ru_m_tgt)):
  row = []
  for i in range(len(ru_m_tgt[u])):
    all_n = suv_t[u]
    users_taking_i = ru_m_tgt[:, i]   # Nu
    sum = 0.0
    for v in range(len(users_taking_i)):
        sum = sum + suv_t[u][v] * users_taking_i
    row.append(sum)
  rating_pred.append(row)

# TWIN

In [ ]:
src_pers = src[['open', 'cons', 'extra', 'agree', 'neuro']]

In [ ]:
df = pd.DataFrame(src_pers)  # NU
df.columns = ['open', 'cons', 'extra', 'agree', 'neuro']
df['userId'] = src.userId
df2 = df.groupby(by='userId').mean().reset_index()
df2 = df2[["open", 'cons', 'extra', 'agree', 'neuro']] 
src_p_arr = np.array(df2)

In [ ]:
arr = src.groupby(by='userId').mean().reset_index()
arr = arr[["open", 'cons', 'extra', 'agree', 'neuro']]

In [ ]:
arr.shape

(988, 5)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
sim = np.abs(1 - euclidean_distances(arr, arr))

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit(arr)

In [ ]:
distances, indices = knn.kneighbors(arr)

In [ ]:
# rating matrix wrt user u
ru_src = src.pivot_table(index='userId',columns='itemId',values='rating')    # U
ru_src = ru_src.fillna(0)
ru_m_src = ru_src > 0
ru_m_src = ru_m_src.replace(True, 1)
ru_m_src = ru_m_src.replace(False, 0)
ru_src = np.array(ru_src)
ru_m_src = np.array(ru_m_src)   # U

In [ ]:
items_src = []    # U
for i in range(len(ru_m_src)):
  row = []
  for j in range(len(ru_m_src[i])):
    if ru_m_src[i][j] == 1:
      row.append(j)
  items_src.append(row)

In [ ]:
ru_m_src

In [ ]:
rating_pred = []
for u in range(len(ru_m_src)):
  row = []
  for i in range(len(ru_m_src[u])):
    sum = 0
    all_n = indices[u]
    users_taking_i = ru_m_src[:, i]   # Nu
    for v in range(len(users_taking_i)):
        sum = sum + np.sum(sim[u][v] * users_taking_i)
        if sum > 5:
          sum = 5 
    row.append(sum)
  rating_pred.append(row)

In [ ]:
pred = np.array(rating_pred)
pred.shape

(988, 12)

In [ ]:
sim[u][v]

1.0

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
mae = mean_absolute_error(ru_src, pred)
rmse = mean_squared_error(ru_src, pred)
print(mae, rmse)

4.664051956815115 23.15038798920378
